In [ ]:
# this will ask you to authenticate with Google
from google.colab import drive
drive.mount('/content/drive')
 
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Uny/DMT/Assignment 2')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

import seaborn as sns
sns.set_style("darkgrid")

In [2]:
#Read the training data from the provided file and save it in the variable dataset as a DataFrame object
training_data = pd.read_csv("data/processed_training_data.csv")
training_data

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,click_bool,booking_bool,score
0,0,1,2013-04-04 08:32:15,12,187,219,893,3,3.5,1,...,1,0,4,0,1,1,1,0,0,0
1,1,1,2013-04-04 08:32:15,12,187,219,10404,4,4.0,1,...,1,0,4,0,1,1,1,0,0,0
2,2,1,2013-04-04 08:32:15,12,187,219,21315,3,4.5,1,...,1,0,4,0,1,1,1,0,0,0
3,3,1,2013-04-04 08:32:15,12,187,219,27348,2,4.0,1,...,1,0,4,0,1,1,1,0,0,0
4,4,1,2013-04-04 08:32:15,12,187,219,29604,4,3.5,1,...,1,0,4,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3470838,3470838,233039,2013-05-19 11:53:45,5,219,219,52908,3,4.0,1,...,4,18,2,1,1,1,0,0,0,0
3470839,3470839,233039,2013-05-19 11:53:45,5,219,219,61253,2,3.0,1,...,4,18,2,1,1,1,0,0,0,0
3470840,3470840,233039,2013-05-19 11:53:45,5,219,219,66013,2,3.0,0,...,4,18,2,1,1,1,0,0,0,0
3470841,3470841,233039,2013-05-19 11:53:45,5,219,219,71380,3,4.0,1,...,4,18,2,1,1,1,0,0,0,0


In [4]:
#Read the test data from the provided file and save it in the variable dataset as a DataFrame object
test_data = pd.read_csv("data/processed_testing_data.csv")
test_data

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,score
0,0,1,2013-02-02 15:27:40,24,216,219,3180,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
1,1,1,2013-02-02 15:27:40,24,216,219,5543,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
2,2,1,2013-02-02 15:27:40,24,216,219,14142,2,3.5,1,...,0,19222,1,10,2,0,1,0,0,0
3,3,1,2013-02-02 15:27:40,24,216,219,22393,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
4,4,1,2013-02-02 15:27:40,24,216,219,24194,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,4959178,332787,2013-05-21 11:06:37,24,216,117,32019,4,3.5,0,...,0,19246,2,7,1,0,1,0,0,0
4959179,4959179,332787,2013-05-21 11:06:37,24,216,117,33959,4,3.0,1,...,0,19246,2,7,1,0,1,0,0,0
4959180,4959180,332787,2013-05-21 11:06:37,24,216,117,35240,4,0.0,0,...,0,19246,2,7,1,0,1,0,0,0
4959181,4959181,332787,2013-05-21 11:06:37,24,216,117,94437,4,0.0,0,...,0,19246,2,7,1,0,1,0,0,0


Benchmarks

In [5]:
def simple_benchmarks(searcher_id, property_id, score, benchmark="simple", seed=1):
    """
    A functions that creates two simple benchmarks:
    - simple: returns the ranking based on the order in which the property is found in the dataset
    - random: returns the ranking based on a random order
    """

    #Creates a list of tuples with the searcher id and property id
    id_and_property = list(zip(searcher_id, property_id, score))

    #If the simple benchmark is selected then the list of tuples will remain unchanged 
    if benchmark == "simple":
        pass
    #If the simple benchmark is selected then the list of tuples will be shuffled in a random order
    #and the name of the file where the prediction will be saved will change
    if benchmark == "random":
        np.random.seed(seed)
        np.random.shuffle(id_and_property)

    return id_and_property

In [6]:
def complex_benchmarks(searcher_id, property_id, review, price, score, benchmark="review"):
    """
    A functions that creates tree complex benchmarks:
    - review: returns the ranking based on the review score for each property, from highest to the lowest
    - price: returns the ranking based on the price of the property, from lowest to the highest
    - both: returns the ranking based on the review score and the price of the property, price(lowest->highest), review(highest->lowest)
    """
    #Creates a list of tuples that contain the searcher id, proerty id, score, review score and property price
    id_and_property = list(zip(searcher_id, property_id, score, review, price))

    #Sorts the ids based on the review score of each property, from highest ot the lowest
    #Set the path where the prediction will be saved
    if benchmark == "review":
        id_and_property.sort(key=lambda i: i[3], reverse=True)

    #Sorts the ids based on the price of each property, from lowes to the highest
    #Set the path where the prediction will be saved
    if benchmark == "price":
        id_and_property.sort(key=lambda i: i[4])

    #Standardize the output of the function so it has the same output as simple benchmark
    id_and_property = [element[:3] for element in id_and_property]
    return id_and_property

Calculate the accuracy of the model

In [7]:
def calculate_dcg(scores):
    return np.sum(scores / np.log2(np.arange(len(scores)) + 2))

In [8]:
def calculate_ndcg(id_property_score):
    score = [element[2] for element in id_property_score]
    ordered_score = sorted(score, reverse=True)
    dcg = calculate_dcg(score)
    idcg = calculate_dcg(ordered_score)
    return dcg/idcg

Test all the benchmarks and see which one of them delivers the best accuracy

In [9]:
unique_training_ids = training_data["srch_id"].unique()
print(unique_training_ids[0])
unique_training_ids = np.delete(unique_training_ids, 0)
unique_training_ids

1


array([     4,      6,      8, ..., 233035, 233036, 233039], dtype=int64)

In [10]:
grouped_training_data = training_data.groupby("srch_id")

In [11]:
simple_total_score = 0
random_total_score = 0
review_total_score = 0
price_total_score = 0

for id in unique_training_ids:
    group_data = grouped_training_data.get_group(id)

    simple_score = simple_benchmarks(group_data["srch_id"], group_data["prop_id"], group_data["score"])
    random_score = simple_benchmarks(group_data["srch_id"], group_data["prop_id"], group_data["score"], benchmark="random")

    review_score = complex_benchmarks(group_data["srch_id"], group_data["prop_id"],group_data["prop_review_score"],group_data["price_usd"], group_data["score"])
    price_score = complex_benchmarks(group_data["srch_id"], group_data["prop_id"],group_data["prop_review_score"],group_data["price_usd"], group_data["score"], benchmark="price")
    
    
    simple_total_score += np.nan_to_num(calculate_ndcg(simple_score))
    random_total_score += np.nan_to_num(calculate_ndcg(random_score))
    review_total_score += np.nan_to_num(calculate_ndcg(review_score))
    price_total_score += np.nan_to_num(calculate_ndcg(price_score))

print("simple score:", simple_total_score / (len(unique_training_ids) + 1))
print("random score:",random_total_score / (len(unique_training_ids) + 1))
print("review score:",review_total_score / (len(unique_training_ids) + 1))
print("price score:",price_total_score / (len(unique_training_ids) + 1))

C:\Users\Helia\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


simple score: 0.2384342097665094
random score: 0.23813100426212533
review score: 0.24200014643018095
price score: 0.2600137708115843


Compute the prediction for the test data with the best baseline

In [12]:
unique_test_ids = test_data["srch_id"].unique()
print(unique_test_ids[0])
unique_test_ids = np.delete(unique_test_ids, 0)
unique_test_ids

1


array([     3,      6,      7, ..., 332785, 332786, 332787], dtype=int64)

In [13]:
grouped_test_data = test_data.groupby("srch_id")

In [14]:
total_score = 0
path = "price_prediction.csv"
group_data = grouped_test_data.get_group(1)
prediction = complex_benchmarks(group_data["srch_id"], group_data["prop_id"],group_data["prop_review_score"],group_data["price_usd"], group_data["score"], benchmark="price")

for id in unique_test_ids:
    group_data = grouped_test_data.get_group(id)

    instance_score = complex_benchmarks(group_data["srch_id"], group_data["prop_id"],group_data["prop_review_score"],group_data["price_usd"], group_data["score"], benchmark="price")
    prediction = np.append(prediction, instance_score, axis=0)
    total_score += np.nan_to_num(calculate_ndcg(instance_score))


print(total_score / (len(unique_test_ids) + 1))
#Create a list only with the searcher id and property id
result = [element[:2] for element in prediction]

 #Save the prediction based on the order of the tuples in the list and the specified name of the file
writer = csv.writer(open(path, "w"), lineterminator="\n")
writer.writerow(("srch_id", "prop_id"))
writer.writerows(result)

C:\Users\Helia\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


0.0


In [16]:
df = pd.read_csv("price_prediction.csv")
df

,srch_id,prop_id
0,1,61632
1,1,128085
2,1,14142
3,1,50162
4,1,37567
...,...,...
4958622,332753,42499
4958623,332753,111432
4958624,332753,130336
4958625,332753,100110
